In [ ]:
import os
from pyspark.sql import SparkSession, types
from pyspark.sql import functions as F, Row

spark = SparkSession.builder.getOrCreate()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mplcursors
import plotly.express as px

In [ ]:
from countryguess import guess_country

# Global Health Data Analysis

TODO: Create a dimension table of country-code and join with relevant fact tables, otherwise it's just too messy

<ol>
    <li>Trend of schizophrenia, BPD, eating disorders, anxiety disorders, drug use disorders, depression, alcohol use disorders for each country over 1990-2019</li>
    <li>Compare between countries in different regions</li>
    <li>For each region/country, which is the fastest growing problem?</li>
    <li>For each problem, which region/country shows the fastest growth?</li>
    <li>Are there any correlations between problems? What countries support that correlation?</li>
    <li>Effects of major events like COVID, war, economic crises etc.</li>
</ol>

### Beginning data exploration bits

In [ ]:
input_data_dir = "Datasets/Global Health Data Analysis 1990-2019/Mental health 2023/CSV"

In [ ]:
print(os.listdir(input_data_dir)) # List of data files

### Mental health depression disorder data

In [ ]:
# Read the file with the headers given, so that we don't have to specify a schema
mhdd_df = spark.read.option("header", True).csv(input_data_dir + "/Mental health Depression disorder Data.csv")

In [ ]:
# Inspect the columns that are there
mhdd_df.printSchema()

In [ ]:
# Take a sample of the first 5 rows of the dataframe
# mhdd_df.show(5, truncate=False)

Transformation 1: There are a lot of weird dates on the data. Some ranging all the way back to BCE. We want data only from 1990-2023 because it is more likely to be relevat and properly documented.

In [ ]:
mhdd_df.select(["Year"]).distinct().show()

In [ ]:
date_filtered_mhdd_df = mhdd_df.filter(mhdd_df["Year"].isin([str(i) for i in range(1990, 2024)])) # Filter for 1990-2023

In [ ]:
date_filtered_mhdd_df.select(["Year"]).distinct().show()

Transformation 2: Inspect and get rid of rows that have values > 100% for any of the percentage occurrences

In [ ]:
erroneous_vals_filtered_mhdd_df = date_filtered_mhdd_df.filter(~((date_filtered_mhdd_df["Schizophrenia (%)"]>100) | (date_filtered_mhdd_df["Bipolar disorder (%)"]>100) | (date_filtered_mhdd_df["Eating disorders (%)"]>100) | (date_filtered_mhdd_df["Drug use disorders (%)"]>100) | (date_filtered_mhdd_df["Depression (%)"]>100) | (date_filtered_mhdd_df["Alcohol use disorders (%)"]>100)))

In [ ]:
erroneous_vals_filtered_mhdd_df.filter(date_filtered_mhdd_df["Entity"]=="Afghanistan").show()

No null values! YAY!

In [ ]:
erroneous_vals_filtered_mhdd_df.filter((date_filtered_mhdd_df["Schizophrenia (%)"].isNull()) | (date_filtered_mhdd_df["Bipolar disorder (%)"].isNull()) | (date_filtered_mhdd_df["Eating disorders (%)"].isNull()) | (date_filtered_mhdd_df["Drug use disorders (%)"].isNull()) | (date_filtered_mhdd_df["Depression (%)"].isNull()) | (date_filtered_mhdd_df["Alcohol use disorders (%)"].isNull())).show()

Transformation 3: Select countries that are present in both datasets after unifying names across the board

In [ ]:
country_fact_schema = types.StructType([
    types.StructField('country_name', types.StringType()),
    types.StructField('country_code', types.StringType()),
    types.StructField('un_region', types.StringType()),
])

In [ ]:
atlas_countries_df = spark.read.csv("Datasets/temp_atlas_outputs/atlas_countries_facts", schema=country_fact_schema)

In [ ]:
atlas_countries_df.show()

In [ ]:
mhdd_df_countries = erroneous_vals_filtered_mhdd_df.select(["Entity"]).distinct().collect() # List of countries in MHDD
mhdd_df_countries_list = [mhdd_df_countries[i]["Entity"] for i in range(len(mhdd_df_countries))]

In [ ]:
print(len(mhdd_df_countries_list))

In [ ]:
atlas_countries_merged = erroneous_vals_filtered_mhdd_df.join(atlas_countries_df.hint("broadcast"), (atlas_countries_df['country_code'] == erroneous_vals_filtered_mhdd_df['Code']))

In [ ]:
atlas_countries_merged.printSchema()

In [ ]:
atlas_countries_merged = atlas_countries_merged.drop("Entity").drop("index").drop("Code")
atlas_countries_merged = atlas_countries_merged.withColumnsRenamed({
    "Year": "year",
    "Schizophrenia (%)": "schizophrenia_%",
    "Bipolar disorder (%)": "bipolar_disorder_%",
    "Eating disorders (%)": "eating_disorders_%",
    "Anxiety disorders (%)": "anxiety_disorders_%",
    "Drug use disorders (%)": "drug_use_disorders_%",
    "Depression (%)": "depression_%",
    "Alcohol use disorders (%)": "alcohol_use_disorders_%",
}).select([
    "country_name",
    "country_code",
    "un_region",
    "year",
    "schizophrenia_%",
    "bipolar_disorder_%",
    "eating_disorders_%",
    "anxiety_disorders_%",
    "drug_use_disorders_%",
    "depression_%",
    "alcohol_use_disorders_%"
])

In [ ]:
atlas_countries_merged.show()

In [ ]:
output_atlas_countries_merged = "Datasets/global_mental_health_csvs/atlas_countries_merged"
atlas_countries_merged.write.csv(output_atlas_countries_merged, mode="overwrite")

### Prevalence of depression (male vs. female)

In [ ]:
dep_mf_df = spark.read.option("header", True).csv(input_data_dir + "/prevalence-of-depression-males-vs-females.csv")

In [ ]:
dep_mf_df.printSchema() # Rename columns

In [ ]:
dep_mf_df.show(5, truncate=False)

In [ ]:
dep_mf_df.select(["Entity"]).distinct().count()

In [ ]:
dep_mf_df.select(["Year"]).distinct().show()

Transformation 1: There are a lot of weird dates on the data. Some ranging all the way back to BCE. We want data only from 1990-2023 because it is more likely to be relevat and properly documented.

In [ ]:
date_filtered_dep_mf_df = dep_mf_df.filter(dep_mf_df["Year"].isin([str(i) for i in range(1990, 2024)])) # Filter for 1990-2023

In [ ]:
date_filtered_dep_mf_df.select(["Year"]).distinct().show()

Transformation 2: Rename columns

In [ ]:
date_filtered_dep_mf_df = date_filtered_dep_mf_df.withColumnsRenamed({
    "Prevalence - Depressive disorders - Sex: Male - Age: Age-standardized (Percent)": "male_depression_prevalence_%",
    "Prevalence - Depressive disorders - Sex: Female - Age: Age-standardized (Percent)": "female_depression_prevalence_%",
    "Population (historical estimates)": "Population"
})

In [ ]:
date_filtered_dep_mf_df.filter(date_filtered_dep_mf_df["Year"]==2017).show()

Transformation 3: Deal with null values

In [ ]:
null_filtered_dep_mf_df = date_filtered_dep_mf_df.filter(~((date_filtered_dep_mf_df["male_depression_prevalence_%"].isNull()) | (date_filtered_dep_mf_df["female_depression_prevalence_%"].isNull())))

In [ ]:
null_filtered_dep_mf_df.filter((null_filtered_dep_mf_df["Entity"]=="Nigeria") & (null_filtered_dep_mf_df["Year"]>2005)).show()

Transformation 4: Merge with countries in Atlas Dataset

In [ ]:
final_dep_mf_df = null_filtered_dep_mf_df.join(atlas_countries_df.hint("broadcast"), (atlas_countries_df['country_code'] == null_filtered_dep_mf_df['Code']))

In [ ]:
final_dep_mf_df.printSchema()

In [ ]:
final_dep_mf_df = final_dep_mf_df.drop("Entity").drop("index").drop("Code").drop("Continent").drop("Population")
final_dep_mf_df = final_dep_mf_df.withColumnsRenamed({
    "Year": "year",
}).select([
    "country_name",
    "country_code",
    "un_region",
    "year",
    "male_depression_prevalence_%",
    "female_depression_prevalence_%"
])

In [ ]:
final_dep_mf_df.show()

In [ ]:
output_dep_mf_atlas_countries_merged = "Datasets/global_mental_health_csvs/dep_mf_atlas_countries_merged"
final_dep_mf_df.write.csv(output_dep_mf_atlas_countries_merged, mode="overwrite")

### Prevalence of eating disorders (male vs. female)

In [ ]:
ed_mf_df = spark.read.option("header", True).csv(input_data_dir + "/prevalence-of-eating-disorders-in-males-vs-females.csv")

In [ ]:
ed_mf_df.printSchema()

In [ ]:
ed_mf_df.show(5, truncate=False)

In [ ]:
ed_mf_df.select(["Entity"]).distinct().count()

In [ ]:
ed_mf_df.select(["Year"]).distinct().show()

Transformation 1: There are a lot of weird dates on the data. Some ranging all the way back to BCE. We want data only from 1990-2023 because it is more likely to be relevat and properly documented.

In [ ]:
date_filtered_ed_mf_df = ed_mf_df.filter(ed_mf_df["Year"].isin([str(i) for i in range(1990, 2024)])) # Filter for 1990-2023

In [ ]:
date_filtered_ed_mf_df.select(["Year"]).distinct().show()

Transformation 2: Rename columns

In [ ]:
date_filtered_ed_mf_df = date_filtered_ed_mf_df.withColumnsRenamed({
    "Prevalence - Eating disorders - Sex: Male - Age: Age-standardized (Percent)": "male_eating_disorders_prevalence_%",
    "Prevalence - Eating disorders - Sex: Female - Age: Age-standardized (Percent)": "female_eating_disorders_prevalence_%",
    "Population (historical estimates)": "population"
})

In [ ]:
date_filtered_ed_mf_df.filter(date_filtered_ed_mf_df["Year"]==2017).show()

Transformation 3: Deal with null values

In [ ]:
null_filtered_ed_mf_df = date_filtered_ed_mf_df.filter(~((date_filtered_ed_mf_df["male_eating_disorders_prevalence_%"].isNull()) | (date_filtered_ed_mf_df["female_eating_disorders_prevalence_%"].isNull())))

In [ ]:
null_filtered_ed_mf_df.filter((null_filtered_ed_mf_df["Entity"]=="Nigeria") & (null_filtered_ed_mf_df["Year"]>2005)).show()

Transformation 4: Merge with Atlas countries

In [ ]:
ed_final_df = null_filtered_ed_mf_df.join(atlas_countries_df.hint("broadcast"), (null_filtered_ed_mf_df["Code"] == atlas_countries_df["country_code"]))

In [ ]:
ed_final_df.select(["Entity"]).distinct().count()

In [ ]:
ed_final_df.printSchema()

In [ ]:
ed_final_df = ed_final_df.drop("index").drop("Code").drop("Entity").drop("Continent").drop("population").withColumnsRenamed({
    "Year": "year",
}).select([
    "country_name",
    "country_code",
    "un_region",
    "year",
    "male_eating_disorders_prevalence_%",
    "female_eating_disorders_prevalence_%",
])

In [ ]:
ed_final_df.show()

In [ ]:
output_ed_mf_atlas_countries_merged = "Datasets/global_mental_health_csvs/ed_mf_atlas_countries_merged"
ed_final_df.write.csv(output_ed_mf_atlas_countries_merged, mode="overwrite")

### Prevalence of anxiety disorders (male vs. female)

In [ ]:
ad_mf_df = spark.read.option("header", True).csv(input_data_dir + "/prevalence-of-anxiety-disorders-males-vs-females.csv")

In [ ]:
ad_mf_df.printSchema()

In [ ]:
ad_mf_df.show(5, truncate=False)

In [ ]:
ad_mf_df.select(["Entity"]).distinct().count()

In [ ]:
ad_mf_df.select(["Year"]).distinct().show()

Transformation 1: There are a lot of weird dates on the data. Some ranging all the way back to BCE. We want data only from 1990-2023 because it is more likely to be relevat and properly documented.

In [ ]:
date_filtered_ad_mf_df = ad_mf_df.filter(ad_mf_df["Year"].isin([str(i) for i in range(1990, 2024)])) # Filter for 1990-2023

In [ ]:
date_filtered_ad_mf_df.select(["Year"]).distinct().show()

Transformation 2: Rename columns

In [ ]:
date_filtered_ad_mf_df = date_filtered_ad_mf_df.withColumnsRenamed({
    "Prevalence - Anxiety disorders - Sex: Male - Age: Age-standardized (Percent)": "male_anxiety_disorders_prevalence_%",
    "Prevalence - Anxiety disorders - Sex: Female - Age: Age-standardized (Percent)": "female_anxiety_disorders_prevalence_%",
    "Population (historical estimates)": "population"
})

In [ ]:
date_filtered_ad_mf_df.filter(date_filtered_ad_mf_df["Year"]==2017).show()

Transformation 3: Deal with null values

In [ ]:
null_filtered_ad_mf_df = date_filtered_ad_mf_df.filter(~((date_filtered_ad_mf_df["male_anxiety_disorders_prevalence_%"].isNull()) | (date_filtered_ad_mf_df["female_anxiety_disorders_prevalence_%"].isNull())))

In [ ]:
null_filtered_ad_mf_df.filter((null_filtered_ad_mf_df["Entity"]=="Nigeria") & (null_filtered_ad_mf_df["Year"]>2005)).show()

Transformation 4: Merge with Atlas countries

In [ ]:
ad_final_df = null_filtered_ad_mf_df.join(atlas_countries_df.hint("broadcast"), (null_filtered_ad_mf_df["Code"] == atlas_countries_df["country_code"]))

In [ ]:
ad_final_df.select(["Entity"]).distinct().count()

In [ ]:
ad_final_df.printSchema()

In [ ]:
ad_final_df = ad_final_df.drop("index").drop("Code").drop("Entity").drop("Continent").drop("population").withColumnsRenamed({
    "Year": "year",
}).select([
    "country_name",
    "country_code",
    "un_region",
    "year",
    "male_anxiety_disorders_prevalence_%",
    "female_anxiety_disorders_prevalence_%",
])

In [ ]:
ad_final_df.show()

In [ ]:
output_ad_mf_atlas_countries_merged = "Datasets/global_mental_health_csvs/ad_mf_atlas_countries_merged"
ad_final_df.write.csv(output_ad_mf_atlas_countries_merged, mode="overwrite")

### Prevalence of schizophrenia (male vs. female)

In [ ]:
sch_mf_df = spark.read.option("header", True).csv(input_data_dir + "/prevalence-of-schizophrenia-in-males-vs-females.csv")

In [ ]:
sch_mf_df.printSchema()

In [ ]:
sch_mf_df.show(5, truncate=False)

In [ ]:
sch_mf_df.select(["Entity"]).distinct().count()

In [ ]:
sch_mf_df.select(["Year"]).distinct().show()

Transformation 1: There are a lot of weird dates on the data. Some ranging all the way back to BCE. We want data only from 1990-2023 because it is more likely to be relevat and properly documented.

In [ ]:
date_filtered_sch_mf_df = sch_mf_df.filter(sch_mf_df["Year"].isin([str(i) for i in range(1990, 2024)])) # Filter for 1990-2023

In [ ]:
date_filtered_sch_mf_df.select(["Year"]).distinct().show()

Transformation 2: Rename columns

In [ ]:
date_filtered_sch_mf_df = date_filtered_sch_mf_df.withColumnsRenamed({
    "Prevalence - Schizophrenia - Sex: Male - Age: Age-standardized (Percent)": "male_schizophrenia_prevalence_%",
    "Prevalence - Schizophrenia - Sex: Female - Age: Age-standardized (Percent)": "female_schizophrenia_prevalence_%",
    "Population (historical estimates)": "population"
})

In [ ]:
date_filtered_sch_mf_df.filter(date_filtered_sch_mf_df["Year"]==2017).show()

Transformation 3: Deal with null values

In [ ]:
null_filtered_sch_mf_df = date_filtered_sch_mf_df.filter(~((date_filtered_sch_mf_df["male_schizophrenia_prevalence_%"].isNull()) | (date_filtered_sch_mf_df["female_schizophrenia_prevalence_%"].isNull())))

In [ ]:
null_filtered_sch_mf_df.filter((null_filtered_sch_mf_df["Entity"]=="Nigeria") & (null_filtered_sch_mf_df["Year"]>2005)).show()

Transformation 4: Merge with Atlas countries

In [ ]:
sch_final_df = null_filtered_sch_mf_df.join(atlas_countries_df.hint("broadcast"), (null_filtered_sch_mf_df["Code"] == atlas_countries_df["country_code"]))

In [ ]:
sch_final_df.select(["Entity"]).distinct().count()

In [ ]:
sch_final_df.printSchema()

In [ ]:
sch_final_df = sch_final_df.drop("index").drop("Code").drop("Entity").drop("Continent").drop("population").withColumnsRenamed({
    "Year": "year",
}).select([
    "country_name",
    "country_code",
    "un_region",
    "year",
    "male_schizophrenia_prevalence_%",
    "female_schizophrenia_prevalence_%",
])

In [ ]:
sch_final_df.show()

In [ ]:
output_sch_mf_atlas_countries_merged = "Datasets/global_mental_health_csvs/sch_mf_atlas_countries_merged"
sch_final_df.write.csv(output_sch_mf_atlas_countries_merged, mode="overwrite")

### Prevalence of bipolar disorder (male vs. female)

In [ ]:
bpd_mf_df = spark.read.option("header", True).csv(input_data_dir + "/prevalence-of-bipolar-disorder-in-males-vs-females.csv")

In [ ]:
bpd_mf_df.printSchema()

In [ ]:
bpd_mf_df.show(5, truncate=False)

In [ ]:
bpd_mf_df.select(["Entity"]).distinct().count()

In [ ]:
bpd_mf_df.select(["Year"]).distinct().show()

Transformation 1: There are a lot of weird dates on the data. Some ranging all the way back to BCE. We want data only from 1990-2023 because it is more likely to be relevat and properly documented.

In [ ]:
date_filtered_bpd_mf_df = bpd_mf_df.filter(bpd_mf_df["Year"].isin([str(i) for i in range(1990, 2024)])) # Filter for 1990-2023

In [ ]:
date_filtered_bpd_mf_df.select(["Year"]).distinct().show()

Transformation 2: Rename columns

In [ ]:
date_filtered_bpd_mf_df = date_filtered_bpd_mf_df.withColumnsRenamed({
    "Prevalence - Bipolar disorder - Sex: Male - Age: Age-standardized (Percent)": "male_bipolar_disorder_prevalence_%",
    "Prevalence - Bipolar disorder - Sex: Female - Age: Age-standardized (Percent)": "female_bipolar_disorder_prevalence_%",
    "Population (historical estimates)": "population"
})

In [ ]:
date_filtered_bpd_mf_df.filter(date_filtered_bpd_mf_df["Year"]==2017).show()

Transformation 3: Deal with null values

In [ ]:
null_filtered_bpd_mf_df = date_filtered_bpd_mf_df.filter(~((date_filtered_bpd_mf_df["male_bipolar_disorder_prevalence_%"].isNull()) | (date_filtered_bpd_mf_df["female_bipolar_disorder_prevalence_%"].isNull())))

In [ ]:
null_filtered_bpd_mf_df.filter((null_filtered_bpd_mf_df["Entity"]=="Nigeria") & (null_filtered_bpd_mf_df["Year"]>2005)).show()

Transformation 4: Merge with Atlas countries

In [ ]:
bpd_final_df = null_filtered_bpd_mf_df.join(atlas_countries_df.hint("broadcast"), (null_filtered_bpd_mf_df["Code"] == atlas_countries_df["country_code"]))

In [ ]:
bpd_final_df.select(["Entity"]).distinct().count()

In [ ]:
bpd_final_df.printSchema()

In [ ]:
bpd_final_df = bpd_final_df.drop("index").drop("Code").drop("Entity").drop("Continent").drop("population").withColumnsRenamed({
    "Year": "year",
}).select([
    "country_name",
    "country_code",
    "un_region",
    "year",
    "male_bipolar_disorder_prevalence_%",
    "female_bipolar_disorder_prevalence_%",
])

In [ ]:
bpd_final_df.show()

In [ ]:
output_bpd_mf_atlas_countries_merged = "Datasets/global_mental_health_csvs/bpd_mf_atlas_countries_merged"
bpd_final_df.write.csv(output_bpd_mf_atlas_countries_merged, mode="overwrite")

### Suicide rate by country

In [ ]:
sui_2023_mf_df = spark.read.option("header", True).csv(input_data_dir + "/suicide-rate-by-country-2023.csv")

In [ ]:
sui_2023_mf_df.printSchema()

In [ ]:
sui_2023_mf_df.show(5, truncate=False)

In [ ]:
sui_2023_mf_df.select(["Country"]).distinct().count()

In [ ]:
# list_of_countries = sui_2023_mf_df.select(["Country"]).distinct().collect()
# countries_list = [list_of_countries[i]["Country"] for i in range(len(list_of_countries))]
# total_countries = set(countries_list)
# country_dict_schema = types.StructType([
#     types.StructField("country", types.StringType()),
# ])
# rows = [Row(row) for row in total_countries]
# country_dict_df = spark.createDataFrame(rows, schema=country_dict_schema)
# country_dict_df = country_dict_df.withColumn("country_code", get_country_code(F.col("Country")))
# country_dict_df.show()
# country_dict_df.write.csv("gmh_sui_2023_country_dict", mode="overwrite")

In [ ]:
# # To create a country_dict_df of country_name, official_name, iso3, un_region to avoid using countryguess
# sui_2023_countries = sui_2023_mf_df.select(["Country"]).distinct().collect()
# sui_2023_countries_list = [sui_2023_countries[i]["Country"] for i in range(len(sui_2023_countries))]
# print(sui_2023_countries_list)

Transformation 1: Rename columns

In [ ]:
sui_2023_mf_df = sui_2023_mf_df.withColumnsRenamed({
    "rate2019both": "all_suicide_%",
    "rate2019male": "male_suicide_%",
    "rate2019female": "female_suicide_%"
}).withColumn("year", F.lit("2019"))

Transformation 2: Remove erroneous values (percentage > 100%)

In [ ]:
erroneous_val_filtered_sui_2023_mf_df = sui_2023_mf_df.filter(~((sui_2023_mf_df["all_suicide_%"] > 100) | (sui_2023_mf_df["male_suicide_%"] > 100) | (sui_2023_mf_df["female_suicide_%"] > 100)))

In [ ]:
erroneous_val_filtered_sui_2023_mf_df.show()

In [ ]:
# @F.udf(returnType=types.StringType())
# def get_country_code(country):
#     return guess_country(country)["iso3"]

In [ ]:
# erroneous_val_filtered_sui_2023_mf_df = erroneous_val_filtered_sui_2023_mf_df.withColumn("code", get_country_code(F.col("country")))

In [ ]:
country_dict_df_schema = types.StructType([
    types.StructField("country_dict_country", types.StringType()),
    types.StructField("country_dict_country_code", types.StringType()),
])
country_dict_df = spark.read.csv("gmh_sui_2023_country_dict", schema=country_dict_df_schema)

In [ ]:
erroneous_val_filtered_sui_2023_mf_df = erroneous_val_filtered_sui_2023_mf_df.join(country_dict_df.hint("broadcast"), (erroneous_val_filtered_sui_2023_mf_df["country"]==country_dict_df["country_dict_country"]), how="left")

In [ ]:
erroneous_val_filtered_sui_2023_mf_df = erroneous_val_filtered_sui_2023_mf_df.drop("country").drop("country_dict_country").withColumnsRenamed({
    "country_dict_country_code": "code"
})

In [ ]:
erroneous_val_filtered_sui_2023_mf_df.count()

In [ ]:
final_sui_2023_mf_df = erroneous_val_filtered_sui_2023_mf_df.join(atlas_countries_df.hint("broadcast"), (erroneous_val_filtered_sui_2023_mf_df["code"]==atlas_countries_df["country_code"]), how="right")

In [ ]:
final_sui_2023_mf_df.select(["country_name"]).distinct().count()

In [ ]:
final_sui_2023_mf_df.printSchema()

In [ ]:
final_sui_2023_mf_df = final_sui_2023_mf_df.drop("country").drop("code").select([
    "country_name",
    "country_code",
    "un_region",
    "year",
    "all_suicide_%",
    "male_suicide_%",
    "female_suicide_%"
])

In [ ]:
final_sui_2023_mf_df.show()

In [ ]:
output_sui_mf_atlas_countries_merged = "Datasets/global_mental_health_csvs/sui_mf_atlas_countries_merged"
final_sui_2023_mf_df.write.csv(output_sui_mf_atlas_countries_merged, mode="overwrite")